<a href="https://colab.research.google.com/github/kennycaiguo/-LearnPytorchWithColab/blob/main/torch_tutorial_lesson2_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self,x):
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')